In [1]:
from pathlib import Path
import matplotlib.pyplot as plt
import librosa
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import h5py
from pathlib import Path
from torch.utils import data
import logging

In [2]:
# Wrapping the dataset with load function
class CustomDataset(data.Dataset):
    def __init__(self, file_path, transform=None):
        super().__init__()
        self.train_data_cache = []
        self.test_data_cache = []
        self.manual_data_cache = []
        self.transform = transform
        self.label_count = [0, 0, 0]
        # Search for all h5 files
        p = Path(file_path)
        files = p.glob('*.h5')
        logging.debug(files)
        for h5dataset_fp in files:
            logging.debug(h5dataset_fp)
            with h5py.File(h5dataset_fp.resolve()) as h5_file:
                # Walk through all groups, extracting datasets
                for gname, group in h5_file.items():
                    k = 0
                    j = 0
                    l = 0
                    if gname == 'referenz':
                        label = 0
                    elif gname == 'spitze':
                        label = 1
                    elif gname == 'grenzflaeche':
                        label = 2

                    logging.debug(group.items())
                    for dname, ds in tqdm(group.items()):
                        if k < 3000:
                            for i in np.split(ds, 4):
                                self.train_data_cache.append([label, torch.tensor(i).unsqueeze(0).type(torch.float32)])
                                k += 1
                        if j < 400:
                            for i in np.split(ds, 4):
                                self.test_data_cache.append([label, torch.tensor(i).unsqueeze(0).type(torch.float32)])
                                j += 1
                        if l < 100:
                            for i in np.split(ds, 4):
                                self.manual_data_cache.append([label, torch.tensor(i).unsqueeze(0).type(torch.float32)])
                                l += 1

                        if k > 3000 and j > 400:
                            break

    def __getitem__(self, index):
        return self.data_cache[index]

    def get_test_data(self):
        return self.test_data_cache

    def get_train_data(self):
        return self.train_data_cache

    def __len__(self):
        return len(self.data_cache)

In [13]:
# Defining the network
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv1d(64, 128, kernel_size=3, padding=1)  # endsize 1536 maxpool 3
        self.maxPool1 = nn.MaxPool1d(3)
        self.maxPool2 = nn.MaxPool1d(3)
        self.maxPool3 = nn.MaxPool1d(3)
        self.maxPool4 = nn.MaxPool1d(3)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        self.relu3 = nn.ReLU()
        self.relu4 = nn.ReLU()
        # self.conv5 = nn.Conv1d(128, 256, kernel_size=3, padding=1) #endsize 1024 maxpool 3
        # self.conv6 = nn.Conv1d(256, 512, kernel_size=3, padding=1) # endsize 512 maxpool 3

        self.fc1 = nn.Linear(384, 3)

    def forward(self, x):
        x = F.max_pool1d(F.relu(self.conv1(x)), 3)
        x = F.max_pool1d(F.relu(self.conv2(x)), 3)
        x = F.max_pool1d(F.relu(self.conv3(x)), 3)
        x = F.max_pool1d(F.relu(self.conv4(x)), 3)
        # x = F.max_pool1d(F.relu(self.conv5(x)),3)
        # x = F.max_pool1d(F.relu(self.conv6(x)),3)
        #logging.debug(x.shape)
        x = torch.flatten(x, 1)
        #logging.debug(x.shape)
        x = F.softmax(self.fc1(x), dim=1)

        return x

In [14]:

# the train loop
def train(dataloader, optimizer, criterion, model):
    model.train()
    running_loss = 0.0
    j = 0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs; data is a list of [labels, inputs]

        inputs = data[1]
        labels = data[0]
        inputs = inputs.to(device)
        labels = labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # log statistics
        running_loss += loss.item()
        if i % 500 == 499:
            logging.debug(f"[{epoch}]Loss: {running_loss / 500} ")


In [15]:
# testing the accuracy on single 1024 snippets
def split_test(dataloader, optimizer, criterion, model):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for labels, inputs in dataloader:
            labels, inputs = labels.to(device), inputs.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            correct += (outputs.argmax(1) == labels).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f" Random Teilstück Error: Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f}")





In [16]:
# looking for cuda device and selecting it if possible
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [17]:
# loading the data
customData = CustomDataset("/home/marcus/Dokumente/entladung/")

100%|█████████████████████████████████████| 3401/3401 [00:00<00:00, 7779.82it/s]


In [18]:
# defining the model and moving it to the correct device
model = Network().to(device)
print(model)
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(total_params)

Network(
  (conv1): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (conv4): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (maxPool1): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (maxPool2): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (maxPool3): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (maxPool4): MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (relu1): ReLU()
  (relu2): ReLU()
  (relu3): ReLU()
  (relu4): ReLU()
  (fc1): Linear(in_features=384, out_features=3, bias=True)
)
33699


In [9]:
# defining loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
# defining train and test sets
train_data = customData.get_train_data()
test_data = customData.get_test_data()
train_dataloader = DataLoader(train_data, batch_size=256, shuffle=True, pin_memory=False, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True, pin_memory=False, num_workers=4)
manual_dataloader = DataLoader(customData.manual_data_cache, batch_size=4, shuffle=False, pin_memory=False, num_workers=4)
    

In [11]:
# training loop
for epoch in range(50):  # loop over the dataset multiple times
    train(train_dataloader, optimizer, criterion, model)
    split_test(test_dataloader, optimizer, criterion, model)
    #test_complete(test_dataloader, optimizer, criterion, model)
print('Finished Training')

 Random Teilstück Error: Accuracy: 88.3%, Avg loss: 0.679858
 Random Teilstück Error: Accuracy: 88.9%, Avg loss: 0.667332
 Random Teilstück Error: Accuracy: 90.4%, Avg loss: 0.643285
 Random Teilstück Error: Accuracy: 91.0%, Avg loss: 0.642901
 Random Teilstück Error: Accuracy: 90.8%, Avg loss: 0.639120
 Random Teilstück Error: Accuracy: 91.7%, Avg loss: 0.637110
 Random Teilstück Error: Accuracy: 91.0%, Avg loss: 0.636783
 Random Teilstück Error: Accuracy: 91.7%, Avg loss: 0.634563
 Random Teilstück Error: Accuracy: 91.6%, Avg loss: 0.634240
 Random Teilstück Error: Accuracy: 92.0%, Avg loss: 0.637142
 Random Teilstück Error: Accuracy: 91.0%, Avg loss: 0.637228
 Random Teilstück Error: Accuracy: 91.2%, Avg loss: 0.635636
 Random Teilstück Error: Accuracy: 90.5%, Avg loss: 0.637802
 Random Teilstück Error: Accuracy: 91.4%, Avg loss: 0.634818
 Random Teilstück Error: Accuracy: 90.8%, Avg loss: 0.637352
 Random Teilstück Error: Accuracy: 92.3%, Avg loss: 0.628618
 Random Teilstück Error:

In [12]:
model.eval()
for labels, inputs in manual_dataloader:
    inputs = inputs.to(device)
    labels = labels.to(device)
    logging.debug(model(inputs), labels)
